In [6]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# creating the Data Frame
df = yf.download('^GSPC', start='1990-08-24', end='2011-08-24');

[*********************100%***********************]  1 of 1 completed


In [8]:
# Creating the empirical return distribution
returns = (df.Open - df.Open.shift(252))/df.Open.shift(252)
distribution = returns.dropna()
print(distribution.mean(),distribution.std())

0.08288017854619155 0.18078483020830713


In [ ]:
# Visualizing the empirical distribution
sorted_dist = pd.DataFrame(distribution.sort_values(ascending=True))
ax = sns.distplot(sorted_dist.Open, kde = False, axlabel = 'Returns', bins = 42)
plt.axvline(0.082, 0,4.0)

In [105]:
# Defining the products
def product(prod_number,distribution): 
    extraction = random.choice(distribution)
    
    if prod_number == 1: #product 103.6 
        product_return = 0.036
        
    elif prod_number == 2: #product 100
        if extraction < 0.0:
            product_return = 0.0
        else:
            product_return = 0.4*extraction
    
    elif prod_number == 3: #product 90
        if extraction < -13.70:
            product_return = -10.0
        else:
            product_return = 0.73*extraction
            
            
    elif prod_number == 4: #product 80
        if extraction < -22.00:
            product_return = -20.0
        else:
            product_return = 0.91*extraction
            
    elif prod_number == 5: #product 0
        product_return = extraction
    
    else:
        print("Please select a product number between 1 and 5")
    
    return extraction,product_return

In [121]:
# Computing empirical mean of products

mean = np.zeros(5)
N = 100000

for i in range(5):
    mean[i] = np.mean([product(i+1, distribution)[1] for l in range(N)])
    
sample_avgs = np.around(np.dot(100,mean), decimals=2) 

In [422]:
# Defining the rule by which a person decides to switch product
def new_choice_utility_myop(means, a = .1):
    results = np.dot(100,np.array([product(i+1,distribution)[1] for i in range(5)]))
    sd = np.abs(results - means)
    utilities = means - np.dot(a * .5,sd**2)
    max_utility = np.argmax(utilities)
    return max_utility

In [450]:
# Now we will simulate the experiment, assuming a sample of 100_000 people, distributed as in figure 3 of the paper
N = 105
percentages = np.array([0.067, 0.21, 0.419, 0.267, 0.038])
holders = np.dot(N,percentages)
holders = np.array([7,22,44,28,4])
np.sum(holders)

105

In [461]:
# Determining the new choices with utilities
new_choices = np.empty((5,5))
for i in range(5):
    a = [new_choice_utility_myop(sample_avgs) for l in range(holders[i])]
    for j in range(5):
        new_choices[i,j] = sum([nc == j for nc in a])

In [464]:
print(new_choices)

[[ 0.  2.  1.  3.  1.]
 [ 1.  2.  2.  8.  9.]
 [ 0.  9.  5. 14. 16.]
 [ 0.  6.  5.  7. 10.]
 [ 0.  1.  0.  2.  1.]]
